# Tasks (100 pts)
1  There may be anomalies in the data that you may need to factor in before you start on the other tasks. Clean the data first to handle these issues. Explain what you did to clean the data (in bulleted form). (10 pt)


2 Compute the Pearson correlation between the following: (9 pt)

> a. Euclidean distance of the ride and the taxi fare

> b. time of day and distance traveled

> c. time of day and the taxi fare

Which has the highest correlation?

3 For each subtask of (2), create a plot visualizing the relation between the variables.

Comment on whether you see non-linear or any other interesting relations. (9 pt)

4 Create an exciting plot of your own using the dataset that you think reveals something

very interesting. Explain what it is, and anything else you learned. (15 pt)

5 Generate additional features like those from (2) from the given data set. What additional

features can you create? (10 pt)

6 Set up a simple linear regression model to predict taxi fare. Use your generated features

from the previous task if applicable. How well/badly does it work? What are the
coefficients for your features? Which variable(s) are the most important one? (12 pt)

7 Consider external datasets that may be helpful to expand your feature set. Give bullet points explaining all the datasets you could identify that would help improve your predictions. If possible, try finding such datasets online to incorporate into your training.

List any that you were able to use in your analysis. (10 pt)

8 Now, try to build a better prediction model that works harder to solve the task. Perhaps it

will still use linear regression but with new features. Perhaps it will preprocess features better (e.g. normalize or scale the input vector, convert non-numerical value into float, or do a special treatment of missing values). Perhaps it will use a different machine learning approach (e.g. nearest neighbors, random forests, etc). Briefly explain what you did differently here versus the simple model. Which of your models minimizes the squared error? (10 pt)

9 Predict all the taxi fares for instances at file “sample_submission.csv”. Write the result into a csv file and submit it to the website. You should do this for every model you develop. Report the rank, score, number of entries, for your highest rank. Include a snapshot of your best score on the leaderboard as confirmation. (15 pt)


In [1]:
# Data processing
import pandas as pd
import numpy as np
import datetime as dt

# Geocoding
import geopy
from geopy.geocoders import Nominatim
from geopy.distance import geodesic as geodesic

# Visualization
import seaborn as sb
import matplotlib.pyplot as plt

# Machine learning
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

# Control Variables

In [2]:
PATH = "./all/"

# read whole file use None
# ROWS = None 
ROWS = 500_000 

TRAIN_FILE = 'train.csv'
TEST_FILE = 'test.csv'

LEFT_BOTTOM_CITY = 'Tennessee'
RIGHT_TOP_CITY = 'Maine'

NYC_LONGITUDE = (-74.256436, -73.699733)
NYC_LATITUDE = (40.495029, 40.915592)

JFK_LONGITUDE = (-73.789185, -73.775291)
JFK_LATITUDE = (40.641960, 40.649562)

In [3]:
geolocator = Nominatim(user_agent='city')
LEFT_BOTTOM_BOUNDARY = geolocator.geocode(LEFT_BOTTOM_CITY)
RIGHT_TOP_BOUNDARY = geolocator.geocode(RIGHT_TOP_CITY)

In [4]:
# Taxicab Rate of Fare
MILEAGE = 0.4 * 5

# Auxiliary Functions

In [5]:
def basic_pearson_corr_result(df, title):
    print('{title}, Pearson correlation coef. of:'.format(title=title))
    format_rule = '    {0:30}'
    # Euclidean distance of the ride and the taxi fare
    print(format_rule.format('dist and fare: '), \
          df['dist'].corr(df['fare_amount']))
    # time of day and distance traveled
    print(format_rule.format('time of day and dist: '), \
          df['hour'].corr(df['dist']))
    # time of day and the taxi fare
    print(format_rule.format('time of day and fare: '), \
          df['hour'].corr(df['fare_amount']))
    
def basic_visualization_result(df):
    df.plot(kind='scatter', x='dist', y='fare_amount')
    df.plot(kind='scatter', x='dist', y='hour')
    df.plot(kind='scatter', x='hour', y='fare_amount')
    
def compare_two_df(df1, title1, df2, title2):
    title_rule = '    {}'
    data_rule = '        {0:30}'

    # Euclidean distance of the ride and the taxi fare
    print('Pearson correlation coef. of:')
    print(title_rule.format('* dist and fare: '))
    print(data_rule.format(title1), \
          df1['dist'].corr(df1['fare_amount']))
    print(data_rule.format(title2), \
          df2['dist'].corr(df2['fare_amount']))

    print(title_rule.format('* time of day and dist: '))
    print(data_rule.format(title1), \
          df1['hour'].corr(df1['dist']))
    print(data_rule.format(title2), \
          df2['hour'].corr(df2['dist']))

    print(title_rule.format('* time of day and fare: '))
    print(data_rule.format(title1), \
          df1['hour'].corr(df1['fare_amount']))
    print(data_rule.format(title2), \
          df2['hour'].corr(df2['fare_amount']))
 
    fig = plt.figure(figsize=(32,32))
    ax1_1 = fig.add_subplot(321)
    ax2_1 = fig.add_subplot(322)
    ax1_2 = fig.add_subplot(323)
    ax2_2 = fig.add_subplot(324)
    ax1_3 = fig.add_subplot(325)
    ax2_3 = fig.add_subplot(326)

    df1.plot(ax=ax1_1, kind='scatter', x='dist', y='fare_amount', title = title1)
    df1.plot(ax=ax1_2, kind='scatter', x='dist', y='hour', title = title1)
    df1.plot(ax=ax1_3, kind='scatter', x='hour', y='fare_amount', title = title1)
    df2.plot(ax=ax2_1, kind='scatter', x='dist', y='fare_amount', title = title2)
    df2.plot(ax=ax2_2, kind='scatter', x='dist', y='hour', title = title2)
    df2.plot(ax=ax2_3, kind='scatter', x='hour', y='fare_amount', title = title2)

## Basic Filter

In [6]:
def valid_fare(row_data):
    """
    this is a soft cut based on initial chare $2.50 and the improvement surcharge 30-cent
    """
    return row_data['fare_amount'] > 2.8

def valid_transportation_region(row_data, left_bottom = LEFT_BOTTOM_BOUNDARY, right_top = RIGHT_TOP_BOUNDARY):
    """
    default region based on the rectangle formed by Tennessee and Maine, 
    since it takes more than 12 hrs to drive from the boundary to NYC or
    from NYC to the boundary
    """
    return (row_data['pickup_longitude'] > left_bottom.longitude) & \
           (row_data['pickup_longitude'] < right_top.longitude) & \
           (row_data['dropoff_longitude'] > left_bottom.longitude) & \
           (row_data['dropoff_longitude'] < right_top.longitude) & \
            (row_data['pickup_latitude'] > left_bottom.latitude) & \
            (row_data['pickup_latitude'] < right_top.latitude) & \
            (row_data['dropoff_latitude'] > left_bottom.latitude) & \
            (row_data['dropoff_latitude'] < right_top.latitude)

def valid_travel_location(row_data):
    """
    pickup and dropoff should not be exactly the same or
    both outside the NYC 
    """
    return (row_data['pickup_longitude'] != row_data['dropoff_longitude']) | \
           (row_data['pickup_latitude'] != row_data['dropoff_latitude']) | \
           (\
            (row_data['pickup_longitude'] >= NYC_LONGITUDE[0]) & \
            (row_data['pickup_longitude'] <= NYC_LONGITUDE[1]) & \
            (row_data['pickup_latitude'] >= NYC_LATITUDE[0]) & \
            (row_data['pickup_latitude'] <= NYC_LATITUDE[1]) \
           ) | (\
            (row_data['dropoff_longitude'] >= NYC_LONGITUDE[0]) & \
            (row_data['dropoff_longitude'] <= NYC_LONGITUDE[1]) & \
            (row_data['dropoff_latitude'] >= NYC_LATITUDE[0]) & \
            (row_data['dropoff_latitude'] <= NYC_LATITUDE[1]) \
           )

def valid_passenger_count(row_data):
    """
    by Taxi rule, the maximum capacity is 6
    """
    return (row_data['passenger_count'] > 0) & (row_data['passenger_count'] <= 6)

# def datetime_parser(x):
#     print(x['pickup_datetime'])
#     try:
#         datetime_object = datetime.strptime(x['pickup_datetime'], '%Y-%m-%d %H:%M:%S')
#         return datetime_object.hours*60 + datetime_object.minutes
#     except:
#         return pd.NaT

## Advanced Filter

In [7]:
def valid_fare_amount_judge_by_travel_distance(row_data):
    return valid_fare_amount_with_long_distance(row_data) & \
           valid_fare_amount_with_short_distance(row_data)

def valid_fare_amount_with_long_distance(row_data):
    return row_data['fare_amount'] >= (row_data['dist'] * MILEAGE + 2.8)

def valid_fare_amount_with_short_distance(row_data):
    return (row_data['dist'] >= 0.2) | (row_data['fare_amount'] <= 10)

def to_jfk(row_data):
    return (row_data['dropoff_longitude'] >= JFK_LONGITUDE[0]) & \
           (row_data['dropoff_longitude'] <= JFK_LONGITUDE[1]) & \
           (row_data['dropoff_latitude'] >= JFK_LATITUDE[0]) & \
           (row_data['dropoff_latitude'] <= JFK_LATITUDE[1])

def remove_outliers_Tukey(usefulData, attr):
    thirdQuartile = usefulData.quantile(.75)[attr]
    firstQuartile = usefulData.quantile(.25)[attr]
    IQR = thirdQuartile - firstQuartile
    return usefulData[usefulData[attr].between(firstQuartile - (IQR * 1.5), thirdQuartile + (IQR * 1.5))]

## Adjust or Create Feature

In [8]:
from math import sin, cos, sqrt, atan2, radians

def dist(row_data):
    R = 6373.0
    s_lon = radians(row_data['pickup_longitude'])
    s_lat = radians(row_data['pickup_latitude'])
    e_lon = radians(row_data['dropoff_longitude'])
    e_lat = radians(row_data['dropoff_latitude'])
    diff_lon = e_lon - s_lon
    diff_lat = e_lat - s_lat

    a = sin(diff_lat / 2)**2 + cos(s_lat) * cos(e_lat) * sin(diff_lon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return R * c * 0.621371

def dist_by_geopy(row_data):
    s_lon = row_data['pickup_longitude']
    s_lat = row_data['pickup_latitude']
    e_lon = row_data['dropoff_longitude']
    e_lat = row_data['dropoff_latitude']
    return geodesic((s_lon, s_lat), (e_lon, e_lat)).miles

# clean_data['dist'] = np.vectorize(dist)(clean_data['pickup_longitude'], clean_data['pickup_latitude'], clean_data['dropoff_longitude'], clean_data['dropoff_latitude'])
# https://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude/19412565

def insert_est_or_edt_time(df):
    df.insert(df.shape[1],'new_york_time',
              df.pickup_datetime.dt.tz_localize('utc').\
              dt.tz_convert('America/New_York'))
    
def insert_year(df):
    df.insert(df.shape[1], 'year', df.new_york_time.dt.year)

def insert_weekday(df):
    df.insert(df.shape[1], 'weekday', df.new_york_time.dt.weekday)

def insert_time_of_day(df):
    df.insert(df.shape[1], 'hour', df.new_york_time.dt.hour + df.new_york_time.dt.minute/60)

# def insert_dist(df, func = dist):
#     euclidean_dist = df.apply(func, axis = 1)
#     df.insert(df.shape[1], 'dist', euclidean_dist)
    
def insert_feature(df, func = None, feature_name = None):
    if not func or not feature_name:
        raise InputError

    new_data = df.apply(func, axis = 1)

    df.insert(df.shape[1], feature_name, new_data)

# Adjust Fare Amout By Rules (http://www.nyc.gov/html/tlc/html/passenger/taxicab_rate.shtml)
```
Metered Fare Information
Onscreen rate is ‘Rate #01 – Standard City Rate.’
The initial charge is $2.50.
Plus 50 cents per 1/5 mile or 50 cents per 60 seconds in slow traffic or when the vehicle is stopped.
In moving traffic on Manhattan streets, the meter should “click” approximately every four downtown blocks, or one block going cross-town (East-West).
There is a 50-cent MTA State Surcharge for all trips that end in New York City or Nassau, Suffolk, Westchester, Rockland, Dutchess, Orange or Putnam Counties.
There is a 30-cent Improvement Surcharge. 
There is a daily 50-cent surcharge from 8pm to 6am.
There is a $1 surcharge from 4pm to 8pm on weekdays, excluding holidays.
Passengers must pay all bridge and tunnel tolls.
Your receipt will show your total fare including tolls. Please take your receipt.
The driver is not required to accept bills over $20.
Please tip your driver for safety and good service.
There are no charges for extra passengers or bags.

To/From JFK and any location in Manhattan:
Onscreen rate is ‘Rate #02 – JFK Airport.’
This is a flat fare of $52 plus tolls, the 50-cent MTA State Surcharge, the 30-cent Improvement Surcharge, and $4.50 rush hour surcharge (4 PM to 8 PM weekdays, excluding legal holidays).
Passenger is responsible for paying all tolls.
Please tip your driver for safety and good service.
```

In [9]:
def adjust_fare(row_data):
    fare = row_data['fare_amount']
    
    if to_jfk(row_data): # should consider from_jfk too 
        # flat fare of $52
        fare -= 52
        # MTA State Surcharge
        fare -= 0.5
        # 30-cent Improvement Surcharge,
        fare -= 0.3
        
        if is_rush(row_data):
            fare -= 4.5
    else:
        pass

# Get Data & Transform UTC to EST or EDT

In [10]:
train_df = pd.read_csv(PATH + TRAIN_FILE, nrows = ROWS)
test_df = pd.read_csv(PATH + TEST_FILE)

train_df['pickup_datetime'] = pd.to_datetime(train_df.pickup_datetime, infer_datetime_format = True)

# insert new_york_time
insert_est_or_edt_time(train_df)

basic_filter_df = train_df[:]
basic_filter_df.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,new_york_time
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21,-73.844311,40.721319,-73.841610,40.712278,1,2009-06-15 13:26:21-04:00
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16,-74.016048,40.711303,-73.979268,40.782004,1,2010-01-05 11:52:16-05:00
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00,-73.982738,40.761270,-73.991242,40.750562,2,2011-08-17 20:35:00-04:00
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42,-73.987130,40.733143,-73.991567,40.758092,1,2012-04-21 00:30:42-04:00
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00,-73.968095,40.768008,-73.956655,40.783762,1,2010-03-09 02:51:00-05:00


# Data Cleaning

## Remove trivial anomaly as follows:
* data['fare_amount'] <= 2.5
* data['longitude'] >= -69.445473 or data['longitude'] <= -86.580444
* data['latitude'] <= 35.517490 or data['latitude'] >= 45.253784
* pickup location == dropoff location
* data['passenger_count'] < 0 or data['passenger_count'] > 6

>[How many people can fit into a yellow taxicab?](#http://www.nyc.gov/html/tlc/html/faq/faq_pass.shtml)

> From Driver Rule 54-15(g) (in PDF):
The maximum amount of passengers allowed in a yellow taxicab by law is four (4) in a four (4) passenger taxicab or five (5) passengers in a five (5) passenger taxicab, except that an additional passenger must be accepted if such passenger is under the age of seven (7) and is held on the lap of an adult passenger seated in the rear.

In [11]:
format_rule = '{0:54}'

print(format_rule.format('Size before data cleaning:'), basic_filter_df.shape[0])
basic_filter_df = basic_filter_df.dropna(how = 'any', axis = 'rows')
print(format_rule.format('Size after removing N/A row data:'), basic_filter_df.shape[0])

# remove invalid fare
basic_filter_df = basic_filter_df.loc[valid_fare(basic_filter_df)]
print(format_rule.format('Size after removing invalid fare:'), basic_filter_df.shape[0])

# remove invalid transportation region
basic_filter_df = basic_filter_df.loc[valid_transportation_region(basic_filter_df)]
print(format_rule.format('Size after removing invalid longitude and latitude:'), basic_filter_df.shape[0])

# remove invalid pickup and dropoff location
basic_filter_df = basic_filter_df.loc[valid_travel_location(basic_filter_df)]
print(format_rule.format('Size after removing invalid travel location:'), basic_filter_df.shape[0])

# remove invalid passenger count
basic_filter_df = basic_filter_df.loc[valid_passenger_count(basic_filter_df)]
print(format_rule.format('Size after removing invalid passenger count:'), basic_filter_df.shape[0])

basic_filter_df.describe()

Size before data cleaning:                             500000
Size after removing N/A row data:                      499995
Size after removing invalid fare:                      497985
Size after removing invalid longitude and latitude:    487954
Size after removing invalid travel location:           487785
Size after removing invalid passenger count:           486042


,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,486042.000000,486042.000000,486042.000000,486042.000000,486042.000000,486042.000000
mean,11.379601,-73.975443,40.750817,-73.974438,40.751211,1.690839
std,9.832618,0.043675,0.031953,0.040399,0.035114,1.306539
min,2.900000,-80.616665,37.419853,-78.659447,37.346533,1.000000
25%,6.000000,-73.992269,40.736561,-73.991569,40.735622,1.000000
50%,8.500000,-73.982084,40.753390,-73.980587,40.753887,1.000000
75%,12.500000,-73.968377,40.767489,-73.965332,40.768420,2.000000
max,500.000000,-69.062927,43.183332,-70.050500,44.600000,6.000000


# Data Preprocessing

In [12]:
insert_year(basic_filter_df)
insert_weekday(basic_filter_df)
insert_time_of_day(basic_filter_df)
insert_feature(basic_filter_df, feature_name = 'dist', func = dist)

basic_filter_df.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,new_york_time,year,weekday,hour,dist
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21,-73.844311,40.721319,-73.841610,40.712278,1,2009-06-15 13:26:21-04:00,2009,0,13.433333,0.640688
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16,-74.016048,40.711303,-73.979268,40.782004,1,2010-01-05 11:52:16-05:00,2010,1,11.866667,5.252316
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00,-73.982738,40.761270,-73.991242,40.750562,2,2011-08-17 20:35:00-04:00,2011,2,20.583333,0.863682
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42,-73.987130,40.733143,-73.991567,40.758092,1,2012-04-21 00:30:42-04:00,2012,5,0.500000,1.739931
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00,-73.968095,40.768008,-73.956655,40.783762,1,2010-03-09 02:51:00-05:00,2010,1,2.850000,1.242608


* Pearson Correlation Coefficients

* Visualization of the Relation between the Variables

* Save feather file

In [13]:
# remove anomaly by using advanced filter
advanced_filter_df = basic_filter_df[valid_fare_amount_judge_by_travel_distance(basic_filter_df)]

In [14]:
basic_filter_df.reset_index().to_feather('./all/basic_filter-' + str(ROWS))
advanced_filter_df.reset_index().to_feather('./all/advanced_filter-' + str(ROWS))

# Simple Linear Regression Model

# Prediction Baseline

In [15]:
# test_df_dist = test_df.apply(dist, axis=1)
# test_df.insert(test_df.shape[1], 'dist', test_df_dist)

In [16]:
# test_X = list(map(lambda x: [x], list(test_df.dist)))

In [17]:
# fare_predict = regr.predict(test_X).round(decimals = 2)

# Create submission file

In [18]:
# submission = pd.DataFrame({'key': test_df.key,\
#                            'fare_amount': fare_predict.ravel()},\
#                           columns = ['key', 'fare_amount'])
# submission.to_csv('submission.csv', index = False)


In [19]:
# submission.shape

# Taxicab Rate of Fare
## Airport Trips
```
* To & From LaGuardia Airport:

This is a metered fare. See Metered Fare information above.
* To/From JFK and any location in Manhattan:

Onscreen rate is ‘Rate 02 – JFK Airport.’
This is a flat fare of $52 plus tolls, the 50-cent MTA State Surcharge, the 30-cent Improvement Surcharge, and $4.50 rush hour surcharge (4 PM to 8 PM weekdays, excluding legal holidays).
Passenger is responsible for paying all tolls.
Please tip your driver for safety and good service.
* From JFK to other New York City destinations:

This is a metered fare. See Metered Fare information above.
* To Newark Airport:

Onscreen rate is ‘Rate 03 – Newark Airport.’
This is a metered fare. See Metered Fare information above.
There is a $17.50 Newark Surcharge.
Passenger is responsible for paying all roundtrip tolls.
There is a 30-cent Improvement Surcharge. 
Please tip your driver for safety and good service.
```

# Reference
[1] https://www1.nyc.gov/nyc-resources/service/1271/yellow-taxi-fares

You can get information about rates and fares for yellow cabs.

Upon entering the taxi, you will be charged the standard City fare rate of $3.30, which includes a 50-cent State surcharge and a 30-cent Improvement surcharge.

Additional charges also apply:

50 cents for every fifth of a mile
50 cents for every minute the taxi traveled less than 12 miles per hour
50 cents night surcharge for travel from 8 PM to 6 AM
$1 for travel from 4 PM to 8 PM on weekdays only
Travel to local airports and trips outside the City are charged different rates.

You can pay by credit or debit cards in yellow taxis with no extra charge.

[2] https://www.codevscolor.com/date-valid-check-python/

[3] https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DatetimeIndex.html

[4] http://stamfordresearch.com/linear-regression-using-pandas-python/